### 환경 불러오기

In [2]:
import pandas as pd
import numpy as np
import os
import requests
import json

### 오피스텔 실거래 데이터 불러오기(2022.03 ~ 2023.02)

In [3]:
# officetels 폴더에 있는 모든 파일명 리스트로 저장
off_forders = os.listdir('../data/officetels')
# csv 파일을 모두 불러와서 병합
df_officetels = pd.read_csv('../data/officetels/'+off_forders[0], encoding='CP949',skiprows=15)
for i in range(1, len(off_forders)):
    if off_forders[i].split('.')[1] == 'csv':
        file = '../data/officetels/'+off_forders[i]
        # csv파일의 불필요한 주석정보 제거
        sub_df = pd.read_csv(file, encoding='CP949',skiprows=15)
        # concat함수로 행을 추가하며 병합
        df_officetels = pd.concat([df_officetels,sub_df],ignore_index=True)
# 도로명 주소가 없는 행은 제거
df_officetels['도로명'].replace(' ',np.nan, inplace=True)
df_officetels.dropna(subset=['도로명'],axis=0,inplace=True)

#### 오피스텔 도로명 주소 Geocoding - 위도, 경도 추출

In [8]:
api_key = '4287bf970222f73d8c88dd3c394bdc99'

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['RoadAddress']
        return float(match_first['x']), float(match_first['y'])
    except:
        return 0, 0

for i in range(0, len(df_officetels)):
    try:
        addr = df_officetels.iloc[i]['도로명']
        x, y = addr_to_lat_lon(addr)
    except:
        x, y = 0, 0
    df_officetels.iloc[i]['longitude'] = x
    df_officetels.iloc[i]['latitude'] = y
    

C:\Users\dhfgo\AppData\Local\Temp\ipykernel_23216\1872086116.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_officetels.iloc[i]['longitude'] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_23216\1872086116.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_officetels.iloc[i]['longitude'] = x
C:\Users\dhfgo\AppData\Local\Temp\ipykernel_23216\1872086116.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_officetels.iloc[

ConnectTimeout: HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EA%B4%91%EB%AA%85%EB%A1%9C%2089 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002EC1968FEB0>, 'Connection to dapi.kakao.com timed out. (connect timeout=None)'))